# The Model

The model to train a CNN model on the MNSIT dataset 

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1,1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1,1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(12544, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    batch_size=64
    test_batch_size=1000
    epochs=3
    lr=1
    gamma=0.7
    no_cuda=False
    seed=1
    log_interval=10
    save_model=False
    
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../mnsit_data', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../mnsit_data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(log_interval, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")
        
    return model


am=main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309748
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.183197
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.757709
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.387961
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.377668
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.491070
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.339782
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.497151
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.311352
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.293243
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.241501
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.236541
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.217612
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.248808
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.242732
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.107098
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.308573
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.396941
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.305113
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.256761
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.005656
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.042796
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.161928
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.038608
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.014865
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.138347
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.163322
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.034710
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.080873
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.087091
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.013742
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.080800
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.043443
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.032750
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.174194
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.074164
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.022293
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.127518
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.139891
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.056968


# Optimal Transport

Calculating the optimal transport distances at different layers


Calculating the first and the second convolutional layers for a data

In [2]:
use_cuda =torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
te=1
if te==1:
    bm = torch.utils.data.DataLoader(
        datasets.MNIST('../mnsit_data', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=5, shuffle=True)
for data,label in bm:
    am.eval()
    data,label=data.to(device), label.to(device)
    x1=am.conv1(data)
    x1=F.relu(x1) # The first covolutional layer 
    x2 = am.conv2(x1)
    x2 = F.relu(x2) #The second convolutional layer
    break
x1=x1.to('cpu').detach().numpy()
x2=x2.to('cpu').detach().numpy()

Generating the cost matrices

In [3]:
import math
import numpy as np
co1=np.zeros((784,784))
for i in range(784):
    for j in range(i,784):
        a=math.floor(i/28)
        b=i%28
        if i==j:
            co1[i][j]=0
        else:
            a1=math.floor(j/28)
            b1=j%28
            q=100*math.sqrt((a-a1)*(a-a1)+(b-b1)*(b-b1))
            co1[i][j]=q
            co1[j][i]=q
co1=co1/co1.max()

In [4]:
def proc(m,eq):
    for i in range(int(eq.size)):
        if eq[i]!=0:
            m[i,:]=m[i,:]/eq[i] 
    return m

def proce(g,E):
    for i in range(576):
        if E[i]!=0:
            g[i,:]=g[i,:]/E[i]
        return g

Calculating the exact distances for the batch of data

In [10]:
import ot
import cv2
import math
m=np.zeros((6,32,784,784))
va1=np.zeros((32,4,4))
z=0
for i in range(4):
    for j in range(i,4):
        if i==j:
            va1[:,i,j]=0
        else:
            e=x1[i].reshape((32,28,28)).astype('float64')
            f=x1[j].reshape((32,28,28)).astype('float64')
            
            for k in range(32):
                sq=e[k].reshape((784))/(np.sum(e[k].reshape((784)),axis=0))
                dis=ot.emd2(sq,f[k].reshape((784))/(np.sum(f[k].reshape((784)),axis=0)),co1)
                mt=ot.emd(e[k].reshape((784))/(np.sum(e[k].reshape((784)),axis=0)),f[k].reshape((784))/(np.sum(f[k].reshape((784)),axis=0)),co1)
                m[z,k,:,:]=proc(mt,sq)
                va1[k][i][j]=dis
                va1[k][j][i]=dis
            z+=1
m=torch.from_numpy(m)
m=m.to(device)
m=m.float()
m = am.conv2(m)
m=m.to('cpu').detach()
m = F.relu(m)
m=m.numpy()
va2=np.zeros((64,4,4))
va2m1=np.zeros((64,4,4))
z=0
for i in range(4):
    for j in range(i,4):
        if i==j:
            va2[:,i,j]=0
        else:
            e=x2[i].reshape((64,28,28)).astype('float64')
            f=x2[j].reshape((64,28,28)).astype('float64')
            print(1)
            for k in range(64):
                sq=e[k].reshape((784))/(np.sum(e[k].reshape((784))))
                io=f[k].reshape((784))/(np.sum(f[k].reshape((784))))
                d1=np.matmul(((m[z,k].reshape((784,784))/np.sum(m[z,k].reshape((784,784)),axis=1)).reshape((784,784))).T,sq).astype('float64')
                d1=d1.reshape((784))/(np.sum(d1.reshape((784)),axis=0))
                dis=ot.emd2(sq,io,co1)
                mt=ot.emd(sq,io,co1)
                dism1=ot.emd2(d1,io,co1)
                va2[k][i][j]=dis
                va2[k][j][i]=dis
                va2m1[k][i][j]=dism1
                va2m1[k][j][i]=dism1
            z+=1
va1=np.sum(va1,axis=0)/32 #averaging it between the number of channels
va2=np.sum(va2,axis=0)/64
va2m1=np.sum(va2m1,axis=0)/64

1


C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Ayush Shrivastav\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in matmul


1
1
1
1
1


In [11]:
va1

array([[0.        , 0.05178904, 0.07642791, 0.0743104 ],
       [0.05178904, 0.        , 0.06829228, 0.05536988],
       [0.07642791, 0.06829228, 0.        , 0.05951704],
       [0.0743104 , 0.05536988, 0.05951704, 0.        ]])

In [12]:
va2

array([[0.        , 0.11215943, 0.13632157, 0.13678228],
       [0.11215943, 0.        , 0.13253972, 0.09634361],
       [0.13632157, 0.13253972, 0.        , 0.12724969],
       [0.13678228, 0.09634361, 0.12724969, 0.        ]])

In [13]:
va2m1

array([[0.        , 0.0864391 , 0.08060525, 0.09026216],
       [0.0864391 , 0.        , 0.08219301, 0.09191341],
       [0.08060525, 0.08219301, 0.        , 0.08872168],
       [0.09026216, 0.09191341, 0.08872168, 0.        ]])